<a href="https://colab.research.google.com/github/Srigowri/uwImageEnhancer/blob/main/UnsupervisedCGAN_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Paper:https://arxiv.org/pdf/1903.09766.pdf

In [ ]:
!pip install albumentations==0.4.6 tqdm --quiet

In [ ]:
import albumentations as A
from tqdm import tqdm
import torch.optim as optim 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset 
from torchvision.utils import save_image
import torchvision
from torchvision import transforms
from google.colab.patches import cv2_imshow
import cv2
from PIL import Image
import os
import numpy as np

from albumentations.pytorch import ToTensorV2

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [ ]:

class DoubleConv(nn.Module):
    def __init__(self,in_channel,out_channel):
        super(DoubleConv,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channel,out_channel,3,1,1,bias=False),  #same input height and widht
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        
            nn.Conv2d(out_channel,out_channel,3,1,1,bias=False),  #same input height and widht
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.conv(x)

class Generator(nn.Module):
    def __init__(self,in_channel=3, out_channel=3,features= [64,128,256,512],):
        super(Generator,self).__init__()
        self.downward  = nn.ModuleList()
        self.upward = nn.ModuleList()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)


        for feature in features:
            self.downward.append(DoubleConv(in_channel,feature))
            in_channel = feature


        for feature in reversed(features):
            self.upward.append(
                nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2)# doubles the height and width

            )
            self.upward.append(DoubleConv(feature*2,feature))

        self.bottleneck = DoubleConv(features[-1],features[-1]*2)

        self.fully_conv = nn.Conv2d(features[0],out_channel,kernel_size=1) #1*1 convolution
        self.tanh = nn.Tanh()

    def forward(self,x):
        
        skip_connect = []
        for down in self.downward:
            x = down(x)   
            skip_connect.append(x)
            x = self.maxpool(x)
        x = self.bottleneck(x)
        

        skip_connect = skip_connect[::-1]

        for idx in range(0,len(self.upward),2):
            x = self.upward[idx](x)

            
            skip_c = skip_connect[idx//2]
            if x.shape!= skip_c.shape:
                x = transforms.functional.resize(x, size = skip_c.shape[2:])

            concat_skip = torch.cat((skip_c,x),dim=1)
            x = self.upward[idx+1](concat_skip)
        
        

        return self.tanh(self.fully_conv(x))


# def test():
#     x = torch.randn((3,3,256,256))

#     model = Generator(in_channel=3,out_channel=3)
#     pred = model(x)

#     print(x.shape)
#     print(pred.shape)
#     assert (pred.shape ==x.shape)
# test()

In [ ]:

class SingleConv(nn.Module):
    def __init__(self,in_channel,out_channel):
        super(SingleConv,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channel,out_channel,3,1,1,bias=False),  #same input height and widht
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        
        )
    def forward(self,x):
        return self.conv(x)
class Discriminator(nn.Module):
    def __init__(self,in_channel=3,out_channel=1,features= [32,64,128,256]):
        super(Discriminator,self).__init__()
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernel_size = 2
        self.downward  = nn.ModuleList()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.downward.append(SingleConv(in_channel,feature))
            in_channel = feature

        self.final =  nn.Conv2d(256,1,1)
        self.sigmoid = nn.Sigmoid()


    def forward(self,x):
        
        for down in self.downward:
            # print(x.size())
            x = down(x)    
            # print(x.size())        
            x = self.maxpool(x)
            # print(x.size())

        x = self.final(x)
        # print(x.size())
        return self.sigmoid(x)

# def test():
#     x = torch.randn((1,3,256,256))
#     y = torch.randn((1,3,256,256))

#     model = Discriminator(in_channel=3,out_channel=1)
#     pred = model(x)


# test()

In [ ]:
#@title
# # @title
# #@title
# L1_Loss = nn.L1Loss()
# MSE = nn.MSELoss()

# # distorted = torch.randn((3,3,256,256)).to(device)
# # undistorted = torch.randn((3,3,256,256)).to(device)



# for i in range(1000):
#     distorted,undistorted = train_data[i]
#     distorted = distorted.unsqueeze(0).to(device)
#     undistorted = undistorted.unsqueeze(0).to(device)

#     generatorU = Generator(in_channel=3,out_channel=3).to(device)
#     generatorU.apply(Weights_Normal)

#     generatorD = Generator(in_channel=3,out_channel=3).to(device)
#     generatorD.apply(Weights_Normal)


#     discriminatorU = Discriminator(in_channel=3,out_channel=1).to(device)
#     discriminatorU.apply(Weights_Normal)


#     discriminatorD = Discriminator(in_channel=3,out_channel=1).to(device)
#     discriminatorD.apply(Weights_Normal)




#     fake_undistorted = generatorU(distorted) 

#     disc_undistorted_real = discriminatorU(undistorted)

#     disc_undistorted_fake = discriminatorU(fake_undistorted.detach())



#     disc_undistorted_real_loss = MSE(disc_undistorted_real,torch.ones_like(disc_undistorted_real))
#     disc_undistorted_fake_loss = MSE(disc_undistorted_fake,torch.zeros_like(disc_undistorted_fake))

#     disc_undistorted_loss = disc_undistorted_real_loss+disc_undistorted_fake_loss


#     fake_distorted = generatorD(undistorted) 


#     disc_distorted_real = discriminatorD(distorted)




#     disc_distorted_fake = discriminatorD(fake_distorted.detach())

#     disc_distorted_real_loss = MSE(disc_distorted_real,torch.ones_like(disc_distorted_real))
#     disc_distorted_fake_loss = MSE(disc_distorted_fake,torch.zeros_like(disc_distorted_fake))

#     disc_distorted_loss = disc_distorted_real_loss+disc_distorted_fake_loss

#     disc_loss = (disc_undistorted_loss + disc_distorted_loss)/2



#     print("jdsfsdfdgdfg")

#     disc_undistorted_fake = discriminatorU(fake_undistorted)
#     print(disc_undistorted_fake.size())
#     disc_distorted_fake =  discriminatorD(fake_distorted)
#     print(disc_distorted_fake.size())

#     gen_undistorted_loss = MSE(disc_undistorted_fake,torch.ones_like(disc_undistorted_fake))
#     gen_distorted_loss = MSE(disc_distorted_fake,torch.ones_like(disc_distorted_fake))

#     #cycle consistent 
#     cycle_distorted = generatorD(fake_undistorted)
#     print(cycle_distorted.size())
#     cycle_undistorted = generatorU(fake_distorted)

#     print(cycle_undistorted.size())

#     cycle_distorted_loss = L1_Loss(distorted,cycle_distorted)
#     cycle_undistorted_loss = L1_Loss(undistorted,cycle_undistorted)

#     ##identity loss 
#     identity_undistorted = generatorD(distorted)
#     print(identity_undistorted.size())
#     identity_distorted = generatorU(undistorted) 
#     print(identity_distorted.size())


#     lambda_cycle  =10
#     gen_loss = (gen_undistorted_loss+gen_distorted_loss+
#                 cycle_distorted_loss*lambda_cycle+
#                 cycle_undistorted_loss*lambda_cycle+
#                 identity_undistorted* lambda_cycle+
#                 identity_distorted*lambda_cycle)
    



In [ ]:
#@title
# #@title
# class VGG19_PercepLoss(nn.Module):
#     """ Calculates perceptual loss in vgg19 space
#     """
#     def __init__(self, _pretrained_=True):
#         super(VGG19_PercepLoss, self).__init__()
#         self.vgg = models.vgg19(pretrained=_pretrained_).features
#         for param in self.vgg.parameters():
#             param.requires_grad_(False)

#     def get_features(self, image, layers=None):
#         if layers is None: 
#             layers = {'30': 'conv5_2'} # may add other layers
#         features = {}
#         x = image
#         for name, layer in self.vgg._modules.items():
#             x = layer(x)
#             if name in layers:
#                 features[layers[name]] = x
#         return features

#     def forward(self, pred, true, layer='conv5_2'):
#         true_f = self.get_features(true)
#         pred_f = self.get_features(pred)
#         return torch.mean((true_f[layer]-pred_f[layer])**2)

In [ ]:
def Weights_Normal(m):
    # initialize weights as Normal(mean, std)
    classname = m.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class UnderWaterDataset(Dataset):
    def __init__(self,trainfolder,transform):
        super(UnderWaterDataset,self).__init__()
        
        self.trainA_path = os.listdir(trainfolder+'trainA')
        self.trainB_path = os.listdir(trainfolder+'trainB')
        
        self.length = min(len(self.trainA_path),len(self.trainB_path))
        self.trainA_path = self.trainA_path[:self.length]
        self.trainB_path = self.trainB_path[:self.length]

        self.transform = transform

    def __len__(self):
        return self.length

    def __getitem__(self,idx):

        
        imgA = np.array(Image.open(trainfolder+"/trainA/"+self.trainA_path[idx]).convert("RGB"))
        imgB = np.array(Image.open(trainfolder+"/trainB/"+self.trainB_path[idx]).convert("RGB"))

    
        imgA = self.transform(image=imgA)['image']
        imgB = self.transform(image=imgB)['image']

        return imgA,imgB


trainfolder = r'/content/drive/MyDrive/CV FINAL PROJECT/EUVP_DATASET/Unpaired/'
testfolder = r'/content/drive/MyDrive/CV FINAL PROJECT/EUVP_DATASET/Test'

img_height,img_width = 256,256

train_transform = A.Compose([
    A.Resize(img_height,img_width),
    A.Rotate(limit=30,p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
    ToTensorV2(),
])

train_data = UnderWaterDataset(trainfolder, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=1)

In [ ]:
generatorD = Generator().to(device)
generatorU = Generator().to(device)

discriminatorD = Discriminator().to(device)
discriminatorU = Discriminator().to(device)

generatorD.apply(Weights_Normal)
generatorU.apply(Weights_Normal)
discriminatorD.apply(Weights_Normal)
discriminatorU.apply(Weights_Normal)

optimizer_G = torch.optim.Adam(list(generatorD.parameters())+list(generatorU.parameters()), lr=2e-4, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(list(discriminatorD.parameters())+list(discriminatorU.parameters()), lr=2e-4, betas=(0.5, 0.999))

L1_Loss = nn.L1Loss()
MSE = nn.MSELoss()

In [ ]:
VAL_DIR = "data/val"
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 4
NUM_EPOCHS = 10
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_GEN_D = "gend.pth.tar"
CHECKPOINT_GEN_U = "genu.pth.tar"
CHECKPOINT_DISC_D = "discd.pth.tar"
CHECKPOINT_DISC_U = "discu.pth.tar"


In [ ]:
!mkdir saved_images

mkdir: cannot create directory ‘saved_images’: File exists


In [ ]:
def train(generatorD,generatorU,discriminatorD,discriminatorU,optimizer_G,optimizer_D,loader,g_scale,d_scaler):
    loop = tqdm(loader,leave=True) #for progress bar
    lambda_cycle= 10
    U_reals = 0
    U_fakes = 0
    for idx, (distorted,undistorted) in enumerate(loop):

        
        distorted = distorted.to(device)
        undistorted = undistorted.to(device)
        

        #train the discriminators first
        
        with torch.cuda.amp.autocast():
            
            fake_undistorted = generatorU(distorted) 
            # print(1,fake_undistorted.size())
            disc_undistorted_real = discriminatorU(undistorted)
            # print(2,disc_undistorted_real.size())
            disc_undistorted_fake = discriminatorU(fake_undistorted.detach())
            # print(3,disc_undistorted_fake.size())

            U_reals += disc_undistorted_real.mean().item()
            U_fakes += disc_undistorted_fake.mean().item()

            disc_undistorted_real_loss = MSE(disc_undistorted_real,torch.ones_like(disc_undistorted_real))
            disc_undistorted_fake_loss = MSE(disc_undistorted_fake,torch.zeros_like(disc_undistorted_fake))

            disc_undistorted_loss = disc_undistorted_real_loss+disc_undistorted_fake_loss

            # print(disc_undistorted_loss)
            
            



            fake_distorted = generatorD(undistorted) 
            # print(4,fake_distorted.size())
            disc_distorted_real = discriminatorD(distorted)
            disc_distorted_fake = discriminatorD(fake_distorted.detach())
            
            disc_distorted_real_loss = MSE(disc_distorted_real,torch.ones_like(disc_distorted_real))
            disc_distorted_fake_loss = MSE(disc_distorted_fake,torch.zeros_like(disc_distorted_fake))

            disc_distorted_loss = disc_distorted_real_loss+disc_distorted_fake_loss
            # print(disc_distorted_loss)

            disc_loss = (disc_undistorted_loss + disc_distorted_loss)/2
            # print(disc_loss)

        optimizer_D.zero_grad()
        d_scaler.scale(disc_loss).backward()
        d_scaler.step(optimizer_D)
        d_scaler.update()


        #train the generators
        with torch.cuda.amp.autocast():
            disc_undistorted_fake = discriminatorU(fake_undistorted)
            disc_distorted_fake =  discriminatorD(fake_distorted)
            gen_undistorted_loss = MSE(disc_undistorted_fake,torch.ones_like(disc_undistorted_fake))
            gen_distorted_loss = MSE(disc_distorted_fake,torch.ones_like(disc_distorted_fake))

            #cycle consistent 
            cycle_distorted = generatorD(fake_undistorted)
            cycle_undistorted = generatorU(fake_distorted)
            
            cycle_distorted_loss = L1_Loss(distorted,cycle_distorted)
            cycle_undistorted_loss = L1_Loss(undistorted,cycle_undistorted)

            ##identity loss 
            # identity_undistorted = generatorD(distorted)
            
            # identity_distorted = generatorU(undistorted) 

            # identity_undistorted_loss = L1_Loss(undistorted, identity_undistorted)
            # identity_distorted_loss = L1_Loss(distorted, identity_distorted)


            gen_loss = (gen_undistorted_loss+gen_distorted_loss+
                      cycle_distorted_loss*lambda_cycle+
                      cycle_undistorted_loss*lambda_cycle)
                    # +identity_undistorted_loss* lambda_cycle+ identity_distorted_loss* lambda_cycle)
            
            
        optimizer_G.zero_grad()
        g_scaler.scale(gen_loss).backward()
        g_scaler.step(optimizer_G)
        g_scaler.update()

        if idx%100 ==0:
            print(f"Discriminator Loss {disc_loss:.2f}, Generator Loss {gen_loss:.2f}")
            save_image(fake_undistorted*0.5+0.5,f"saved_images/undistorted_{idx}.png")
            save_image(fake_distorted*0.5+0.5,f"saved_images/distorted_{idx}.png")
            
            
            
        loop.set_postfix(U_real=U_reals/(idx+1), U_fake=U_fakes/(idx+1))

In [ ]:
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

save = True
for epoch in range(10):
    !cp -r saved_images /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results
    train(generatorD,generatorU,discriminatorD,discriminatorU,optimizer_G,optimizer_D,train_loader,g_scaler,d_scaler)
    

    if epoch%4==0:
        torch.save( {'state_dict': generatorD.state_dict(),'optimizer': optimizer_G.state_dict()},CHECKPOINT_GEN_D )
        torch.save( {'state_dict': generatorU.state_dict(),'optimizer': optimizer_G.state_dict()},CHECKPOINT_GEN_U)
        torch.save( {'state_dict': discriminatorD.state_dict(),'optimizer': optimizer_D.state_dict()}, CHECKPOINT_DISC_D)
        torch.save( {'state_dict': discriminatorU.state_dict(),'optimizer': optimizer_D.state_dict()},CHECKPOINT_DISC_U )
        !cp -r discd.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/discd.pth.tar
        !cp -r discu.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/discu.pth.tar
        !cp -r gend.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/gend.pth.tar
        !cp -r genu.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/genu.pth.tar



  0%|          | 0/393 [00:00<?, ?it/s]

Discriminator Loss 0.51, Generator Loss 10.05




  0%|          | 0/393 [00:04<?, ?it/s, U_fake=0.581, U_real=0.576]

  0%|          | 1/393 [00:04<26:22,  4.04s/it, U_fake=0.581, U_real=0.576]

  0%|          | 1/393 [00:07<26:22,  4.04s/it, U_fake=0.563, U_real=0.561]

  1%|          | 2/393 [00:07<25:16,  3.88s/it, U_fake=0.563, U_real=0.561]

  1%|          | 2/393 [00:11<25:16,  3.88s/it, U_fake=0.549, U_real=0.559]

  1%|          | 3/393 [00:11<24:51,  3.82s/it, U_fake=0.549, U_real=0.559]

  1%|          | 3/393 [00:15<24:51,  3.82s/it, U_fake=0.537, U_real=0.555]

  1%|          | 4/393 [00:15<25:19,  3.91s/it, U_fake=0.537, U_real=0.555]

  1%|          | 4/393 [00:19<25:19,  3.91s/it, U_fake=0.53, U_real=0.55]  

  1%|▏         | 5/393 [00:19<25:17,  3.91s/it, U_fake=0.53, U_real=0.55]

  1%|▏         | 5/393 [00:24<25:17,  3.91s/it, U_fake=0.523, U_real=0.549]

  2%|▏         | 6/393 [00:24<27:25,  4.25s/it, U_fake=0.523, U_real=0.549]

  2%|▏         | 6/393 [00:27<27:25,  4.25s/it, U_fake=0.521, U_real=0.547]

  2%|▏ 

Discriminator Loss 0.47, Generator Loss 3.31




 25%|██▌       | 100/393 [06:32<19:37,  4.02s/it, U_fake=0.487, U_real=0.534]

 26%|██▌       | 101/393 [06:32<19:58,  4.11s/it, U_fake=0.487, U_real=0.534]

 26%|██▌       | 101/393 [06:35<19:58,  4.11s/it, U_fake=0.487, U_real=0.534]

 26%|██▌       | 102/393 [06:35<18:37,  3.84s/it, U_fake=0.487, U_real=0.534]

 26%|██▌       | 102/393 [06:39<18:37,  3.84s/it, U_fake=0.487, U_real=0.533]

 26%|██▌       | 103/393 [06:39<18:16,  3.78s/it, U_fake=0.487, U_real=0.533]

 26%|██▌       | 103/393 [06:43<18:16,  3.78s/it, U_fake=0.487, U_real=0.533]

 26%|██▋       | 104/393 [06:43<18:54,  3.92s/it, U_fake=0.487, U_real=0.533]

 26%|██▋       | 104/393 [06:46<18:54,  3.92s/it, U_fake=0.487, U_real=0.533]

 27%|██▋       | 105/393 [06:46<17:27,  3.64s/it, U_fake=0.487, U_real=0.533]

 27%|██▋       | 105/393 [06:50<17:27,  3.64s/it, U_fake=0.487, U_real=0.533]

 27%|██▋       | 106/393 [06:50<17:51,  3.73s/it, U_fake=0.487, U_real=0.533]

 27%|██▋       | 106/393 [06:54<17:51,  3.73s/it, 

...



 25%|██▌       | 100/393 [01:42<04:50,  1.01it/s, U_fake=0.49, U_real=0.51] 

 26%|██▌       | 101/393 [01:42<05:27,  1.12s/it, U_fake=0.49, U_real=0.51]

 26%|██▌       | 101/393 [01:43<05:27,  1.12s/it, U_fake=0.491, U_real=0.51]

 26%|██▌       | 102/393 [01:43<05:15,  1.08s/it, U_fake=0.491, U_real=0.51]

 26%|██▌       | 102/393 [01:44<05:15,  1.08s/it, U_fake=0.491, U_real=0.51]

 26%|██▌       | 103/393 [01:44<05:06,  1.06s/it, U_fake=0.491, U_real=0.51]

 26%|██▌       | 103/393 [01:45<05:06,  1.06s/it, U_fake=0.491, U_real=0.51]

 26%|██▋       | 104/393 [01:45<04:59,  1.04s/it, U_fake=0.491, U_real=0.51]

 26%|██▋       | 104/393 [01:46<04:59,  1.04s/it, U_fake=0.49, U_real=0.51] 

 27%|██▋       | 105/393 [01:46<04:54,  1.02s/it, U_fake=0.49, U_real=0.51]

 27%|██▋       | 105/393 [01:47<04:54,  1.02s/it, U_fake=0.491, U_real=0.511]

 27%|██▋       | 106/393 [01:47<04:51,  1.02s/it, U_fake=0.491, U_real=0.511]

 27%|██▋       | 106/393 [01:48<04:51,  1.02s/it, U_fake=0.491

Discriminator Loss 0.49, Generator Loss 2.08




 51%|█████     | 200/393 [03:22<03:12,  1.00it/s, U_fake=0.491, U_real=0.51]

 51%|█████     | 201/393 [03:22<03:36,  1.13s/it, U_fake=0.491, U_real=0.51]

 51%|█████     | 201/393 [03:23<03:36,  1.13s/it, U_fake=0.491, U_real=0.51]

 51%|█████▏    | 202/393 [03:23<03:28,  1.09s/it, U_fake=0.491, U_real=0.51]

 51%|█████▏    | 202/393 [03:24<03:28,  1.09s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 203/393 [03:24<03:22,  1.07s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 203/393 [03:25<03:22,  1.07s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 204/393 [03:25<03:17,  1.05s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 204/393 [03:26<03:17,  1.05s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 205/393 [03:26<03:13,  1.03s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 205/393 [03:27<03:13,  1.03s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 206/393 [03:27<03:11,  1.02s/it, U_fake=0.491, U_real=0.51]

 52%|█████▏    | 206/393 [03:28<03:11,  1.02s/it, U_fake=0.491

Discriminator Loss 0.50, Generator Loss 1.90




 76%|███████▋  | 300/393 [05:03<01:32,  1.00it/s, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 301/393 [05:03<01:44,  1.13s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 301/393 [05:04<01:44,  1.13s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 302/393 [05:04<01:39,  1.10s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 302/393 [05:05<01:39,  1.10s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 303/393 [05:05<01:35,  1.06s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 303/393 [05:06<01:35,  1.06s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 304/393 [05:06<01:32,  1.04s/it, U_fake=0.491, U_real=0.511]

 77%|███████▋  | 304/393 [05:07<01:32,  1.04s/it, U_fake=0.491, U_real=0.511]

 78%|███████▊  | 305/393 [05:07<01:30,  1.03s/it, U_fake=0.491, U_real=0.511]

 78%|███████▊  | 305/393 [05:08<01:30,  1.03s/it, U_fake=0.491, U_real=0.511]

 78%|███████▊  | 306/393 [05:08<01:28,  1.02s/it, U_fake=0.491, U_real=0.511]

 78%|███████▊  | 306/393 [05:09<01:28,  1.02s/it, 

In [ ]:
!cp -r saved_images /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results

In [ ]:
!cp -r discd.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/discd.pth.tar
!cp -r discu.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/discu.pth.tar
!cp -r gend.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/gend.pth.tar
!cp -r genu.pth.tar /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results/genu.pth.tar

In [ ]:
class UnderWaterDatasetTest(Dataset):
    def __init__(self,trainfolder,transform):
        super(UnderWaterDatasetTest,self).__init__()
        self.test_path = os.listdir(testfolder)
        self.transform = transform

    def __len__(self):
        return len(self.test_path)

    def __getitem__(self,idx):
        img = np.array(Image.open(testfolder+self.test_path[idx]).convert("RGB"))
        img = self.transform(image=img)['image']
        return img


testfolder = r'/content/drive/MyDrive/CV FINAL PROJECT/EUVP_DATASET/Test/'

img_height,img_width = 256,256

test_transform = A.Compose([
    A.Resize(img_height,img_width),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
    ToTensorV2(),
])

test_data = UnderWaterDatasetTest(testfolder, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, num_workers=1)

modelD = torch.load(r'/content/drive/MyDrive/CV FINAL PROJECT/Results/gend.pth.tar')
modelU = torch.load(r'/content/drive/MyDrive/CV FINAL PROJECT/Results/genu.pth.tar')


generatorD = Generator().to(device)
generatorU = Generator().to(device)

optimizer_G = torch.optim.Adam(list(generatorD.parameters())+list(generatorU.parameters()), lr=2e-4, betas=(0.5, 0.999))


generatorU.load_state_dict(modelU['state_dict'])
optimizer_G.load_state_dict(modelU['optimizer'])


L1_Loss = nn.L1Loss()
MSE = nn.MSELoss()

In [ ]:
!mkdir test_imgs
generatorU.eval()
for idx,x in enumerate(test_loader):
    x = x.to(device)
    undistorted_img = generatorU(x)
    save_image(x*0.5+0.5,f"test_imgs/test_input_{idx}.png")
    save_image(undistorted_img*0.5+0.5,f"test_imgs/test_out_{idx}.png")

!cp -r test_imgs /content/drive/MyDrive/CV\ FINAL\ PROJECT/Results